In [1]:
# from google.colab import drive
# drive.mount("/content/mydrive/")

In [2]:
# cd /content/mydrive/MyDrive/hmdb51/hmdb51

In [ ]:
import tensorflow as tf
import numpy as np
import random
import cv2
import os,sys
from PIL import Image
import scipy.misc


def ToImg(raw_flow,bound):
    flow = np.array(raw_flow, dtype=np.float32)
    flow[flow>bound] = bound
    flow[flow<-bound] = -bound
    flow -= -bound
    flow *= (1/float(2*bound))
    return flow

def save_flows(flows, bound):
    flow_x = np.expand_dims(ToImg(flows[...,0], bound), axis=-1)
    flow_y = np.expand_dims(ToImg(flows[...,1], bound), axis=-1)
    flow_0 = np.zeros_like(flow_x)
    return np.concatenate((flow_x, flow_y, flow_0), axis=-1)


def capture_trim_resize(path, shape, n_frames):
  out = []
  cap = cv2.VideoCapture(path)
  valid = True
  while 1:#for i in range(n_frames):
    _, frame = cap.read()
    if not _:
      break
      frame = np.zeros((shape[0], shape[1], 3), dtype=np.uint8)
    frame = cv2.resize(frame, (shape[1], shape[0]))
    out.append(frame)
  return np.array(out)


def tvl(frames, bound):
  output = []
  len_frame=len(frames)
  frame_num = 0
  image, prev_image, gray, prev_gray = None, None, None, None
  num0 = 0
  while True:
    if num0 >= len_frame:
      break
    frame = frames[num0]
    frame = cv2.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2))
    num0 += 1
    if frame_num == 0:
      image = np.zeros_like(frame)
      gray = np.zeros_like(frame)
      prev_gray = np.zeros_like(frame)
      prev_image = frame
      prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)
      frame_num += 1

    image = frame
    gray=cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    frame_0 = prev_gray
    frame_1 = gray
    dtvl1 = cv2.optflow.DualTVL1OpticalFlow_create()
    flowDTVL1 = dtvl1.calc(frame_0, frame_1, None)
    flow = save_flows(flowDTVL1, bound)
    output.append(flow)
    prev_gray = gray
    prev_image = image
    frame_num += 1
  output = np.array(output)
  return tf.image.resize(output, [frames.shape[1], frames.shape[2]]).numpy()


class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, videos_folder_path, train_list_path, frame_shape, n_frames, batch_size, shuffle=True, flow_bound=15):
    self.path = videos_folder_path
    self.n_frames = n_frames
    self.shape = frame_shape
    self.batch_size = batch_size
    self.shuffle = shuffle
    file = open(train_list_path).readlines()
    self.videos = [self.path + "/" + i.split()[0] for i in file]
    self.labels = [int(i.split()[1]) for i in file]
    self.flow_bound = flow_bound
    self.on_epoch_end()
    
  def on_epoch_end(self):
    if self.shuffle:
      temp = [i for i in range(len(self.labels))]
      random.shuffle(temp)
      self.videos = [self.videos[i] for i in temp]
      self.labels = [self.labels[i] for i in temp]
    

  def __getitem__(self, index):
    start_index = index * self.batch_size
    rgb = []
    flow = []
    y = []
    i = start_index - 1
    while len(y) < self.batch_size:
      try:
        i += 1
        print(self.videos[i % len(self.videos)], self.labels[i % len(self.videos)])
        video = capture_trim_resize(self.videos[i % len(self.videos)], self.shape, self.n_frames)
        op = tvl(video.copy(), self.flow_bound)
        rgb.append(video.astype(np.float32) / 255)
        flow.append(op)
        y.append(self.labels[i % len(self.videos)])
      except Exception as err:
        print(err)
        continue
    return {"input_rgb":np.array(rgb), "input_flow":np.array(flow)}, np.array(y, dtype=np.float32)

  def __len__(self):
    return int(np.floor(len(self.labels) / self.batch_size))

In [ ]:
gen = DataGenerator("videos", "testTrainMulti_7030_splits/trainlist01.txt", (224, 224), 64, 1, False)
gen2 = DataGenerator("videos", "testTrainMulti_7030_splits/testlist01.txt", (224, 224), 64, 1, False)

In [ ]:
classes = os.listdir("videos")
classes.sort()
all_videos = []
all_labels = []
for i in range(len(classes)):
  videos = os.listdir("videos/" + classes[i])
  for j in videos:
    v = "videos/" + classes[i] + "/" + j
    if v not in gen.videos and v not in gen2.videos:
      all_videos.append(v)
      all_labels.append(i + 1)

In [ ]:
gen.videos = all_videos
gen.labels = all_labels
print(len(gen.videos), len(gen.labels))

1666 1666


In [ ]:
size = (224, 224)
for i in range(800, 1000):
  a = gen.__getitem__(i)
  for i in range(len(a[1])):
    print(a[0]["input_rgb"].shape)
    print(a[0]["input_flow"].shape)
    print(a[1])
  c = str(int(a[1][0]))
  f = str(int(len(os.listdir("dataset3/" + c))))
  result = cv2.VideoWriter('dataset3/' + c + '/rgb_' + f + '.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)
  for i in range(1):  
    for j in range(len(a[0]["input_rgb"][0])):
      frame = np.array(a[0]["input_rgb"][i][j] * 255, dtype=np.uint8)
      result.write(frame)
  result.release()
  result = cv2.VideoWriter('dataset3/' + c + '/flow_' + f + '.avi', 
                          cv2.VideoWriter_fourcc(*'MJPG'),
                          10, size)
  for i in range(1):    
    for j in range(len(a[0]["input_rgb"][0])):
      frame = np.array(a[0]["input_flow"][i][j] * 255, dtype=np.uint8)
      result.write(frame)
  result.release()

videos/shoot_gun/Glory_shoot_gun_u_nm_np2_le_med_57.avi 37
(1, 55, 224, 224, 3)
(1, 55, 224, 224, 3)
[37.]
videos/shoot_gun/DefensivePistolShootingTechniques_shoot_gun_u_nm_np1_fr_med_2.avi 37
(1, 98, 224, 224, 3)
(1, 98, 224, 224, 3)
[37.]
videos/shoot_gun/GLOCK17_shoot_gun_f_nm_np1_fr_med_3.avi 37
(1, 110, 224, 224, 3)
(1, 110, 224, 224, 3)
[37.]
videos/sit/20060723sfjffsomelikeitwarmed_sit_u_nm_np1_fr_goo_1.avi 38
(1, 78, 224, 224, 3)
(1, 78, 224, 224, 3)
[38.]
videos/sit/A_Beautiful_Mind_5_sit_u_cm_np1_ri_med_1.avi 38
(1, 91, 224, 224, 3)
(1, 91, 224, 224, 3)
[38.]
videos/sit/AboutABoy_sit_u_cm_np1_ri_med_1.avi 38
(1, 79, 224, 224, 3)
(1, 79, 224, 224, 3)
[38.]
videos/sit/AmericanGangster_sit_f_cm_np1_le_bad_11.avi 38
(1, 48, 224, 224, 3)
(1, 48, 224, 224, 3)
[38.]
videos/sit/AmericanGangster_sit_u_cm_np1_ba_bad_16.avi 38
(1, 110, 224, 224, 3)
(1, 110, 224, 224, 3)
[38.]
videos/sit/AmericanGangster_sit_u_cm_np1_fr_med_26.avi 38
(1, 78, 224, 224, 3)
(1, 78, 224, 224, 3)
[38.]
videos